In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import dhs_util
from dhs_util import *

os.chdir('C:/Users/christian fink/OneDrive/Math485')
df = pd.read_csv('dhs_service_records_synthesized_final.csv')

df = dhs_preprocessing(df)
df, service_map = add_service_label(df)
df = add_age_bin(df)

recipient = get_recipient_attribute(df)

In [9]:
serv_list = []
for groups in df.groupby('id').groups.values():
    serv_list.append(df.loc[groups]['serv'].tolist())

In [10]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax
from mlxtend.frequent_patterns import hmine

In [12]:
def oneHotCoding(serv_list):
    te = TransactionEncoder()
    te_ary = te.fit(serv_list).transform(serv_list)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

serv_oneHot = oneHotCoding(serv_list)
df_id_serv = serv_oneHot
df_id_serv.iloc[:,1:23] = df_id_serv.iloc[:,1:23] > 0


,support,itemsets
4,0.941422,(S12)
2,0.153844,(S09)
14,0.139131,"(S12, S09)"
15,0.103528,"(S09, S14)"
6,0.103528,(S14)
25,0.094436,"(S12, S09, S14)"
20,0.094436,"(S12, S14)"
3,0.040882,(S11)
18,0.032106,"(S12, S11)"
10,0.031396,(S19)


In [96]:
frequent_itemsets1 = apriori(df_id_serv.iloc[:, 1:23], use_colnames=True, min_support=0.01)\
    .sort_values(by="support", ascending=False)
frequent_itemsets2 = fpgrowth(df_id_serv.iloc[:, 1:23], use_colnames=True, min_support=0.0001)\
    .sort_values(by="support", ascending=False)
frequent_itemsets3 = hmine(serv_oneHot,min_support=0.0001,use_colnames=True)\
    .sort_values(by="support", ascending = False)
frequent_itemsets4 = fpmax(serv_oneHot, use_colnames=True, min_support=0.0001)\
    .sort_values(by="support", ascending=False)


#ASKED AI TO HELP WITH DISPLAYING - STILL PREFER TO DO THEM ONE AT A TIME
#frequent_itemsets1.reset_index(drop=True, inplace=True)
#frequent_itemsets2.reset_index(drop=True, inplace=True)
#frequent_itemsets3.reset_index(drop=True, inplace=True)
#frequent_itemsets4.reset_index(drop=True, inplace=True)

#frequent_itemsets1 = frequent_itemsets1.add_prefix('Apriori_')
#frequent_itemsets2 = frequent_itemsets2.add_prefix('FPGrowth_')
#frequent_itemsets3 = frequent_itemsets3.add_prefix('HMine_')
#frequent_itemsets4 = frequent_itemsets4.add_prefix('FPMax_')

pd.concat([frequent_itemsets1, frequent_itemsets2, frequent_itemsets3, frequent_itemsets4], join = 'outer', axis = 1)
#pd.join? or pd.merge?

,support,itemsets,support,itemsets,support,itemsets,support,itemsets
4,0.941422,(S12),0.013687,(S03),0.000146,"(S02, S03, S04)",0.000131,"(S12, S03, S04, S14, S09)"
2,0.153844,(S09),0.040882,(S11),0.004191,(S02),0.000124,"(S12, S22)"
14,0.139131,"(S12, S09)",0.006398,(S07),0.002248,"(S09, S02, S03)",0.000161,"(S12, S03, S02, S14, S05, S09)"
15,0.103528,"(S09, S14)",0.012649,(S05),0.00015,"(S09, S02, S03, S10)",0.000101,"(S19, S10)"
6,0.103528,(S14),0.024002,(S15),0.000817,"(S05, S02, S03)",0.000124,"(S20, S19)"
...,...,...,...,...,...,...,...,...
219,NaN,NaN,0.000101,"(S12, S18, S21, S14, S16, S09)",0.000146,"(S09, S12, S06, S18)",NaN,NaN
439,NaN,NaN,0.000101,"(S19, S12, S21)",0.000536,"(S15, S07, S14)",NaN,NaN
94,NaN,NaN,0.000101,"(S10, S07)",0.000135,"(S04, S03, S14)",NaN,NaN
933,NaN,NaN,NaN,NaN,0.01326,(S21),NaN,NaN


In [ ]:
#THE METHOD I PREFER. I can just remove the '#' from any to make them print,or print them all together
print(frequent_itemsets1)
#print(frequent_itemsets2)
#print(frequent_itemsets3)
#print(frequent_itemsets4)

In [43]:
length_1_itemsets1 = frequent_itemsets1[frequent_itemsets1['Apriori_itemsets'].apply(lambda x: len(x) == 1)]

hmine_l2 = hmine(serv_oneHot,min_support=0.005,use_colnames=True)\
    .sort_values(by="support", ascending = False)-
length_2_hmine = hmine_l2[hmine_l2['itemsets'].apply(lambda x: len(x) == 2)]
length_2_hmine = length_2_hmine.add_prefix('HMine_')

fpmax_l5 = fpmax(serv_oneHot,min_support=0.0001,use_colnames=True)\
    .sort_values(by="support", ascending = False)
length_5_fpmax = fpmax_l5[fpmax_l5['itemsets'].apply(lambda x: len(x) == 5)]
length_5_fpmax = length_5_fpmax.add_prefix('FPMax_')


print(length_1_itemsets1)
print(length_2_hmine)
print(length_5_fpmax)


    Apriori_support Apriori_itemsets
0          0.941422            (S12)
1          0.153844            (S09)
4          0.103528            (S14)
7          0.040882            (S11)
9          0.031396            (S19)
10         0.024002            (S15)
16         0.013687            (S03)
17         0.013573            (S18)
21         0.013260            (S21)
22         0.012967            (S17)
23         0.012649            (S05)
25         0.011915            (S13)
   HMine_support HMine_itemsets
16      0.139131     (S12, S09)
28      0.103528     (S09, S14)
46      0.094436     (S12, S14)
41      0.032106     (S12, S11)
50      0.022561     (S12, S15)
32      0.019431     (S09, S15)
12      0.018307     (S09, S11)
35      0.013468     (S09, S18)
60       0.01346     (S18, S14)
3       0.012284     (S12, S03)
45      0.011448     (S12, S13)
53      0.010384     (S12, S18)
54      0.009949     (S19, S12)
58      0.009888     (S15, S14)
43      0.009374     (S11, S14)
33     

In [108]:
#freq_itemset_fpgrowth = fpgrowth(serv_oneHot,min_support=0.05,use_colnames=True)
association_rulesfpgrowth = association_rules(frequent_itemsets2, metric="confidence", min_threshold = 0.5)
fpgrowth_rules = association_rulesfpgrowth.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'lift']]
fpgrowth_rules


,antecedents,consequents,support,lift
1293,"(S16, S12, S09, S07)","(S11, S18, S14)",0.000253,460.289122
1252,"(S16, S12, S07)","(S11, S18, S14)",0.000253,460.289122
1301,"(S16, S12, S07)","(S09, S11, S18, S14)",0.000253,460.289122
1235,"(S16, S12, S07)","(S09, S11, S18)",0.000253,459.842673
1209,"(S16, S12, S07)","(S11, S18)",0.000253,459.842673
...,...,...,...,...
2127,"(S16, S19, S09)",(S12),0.000142,0.593595
2130,"(S16, S19, S14)",(S12),0.000142,0.593595
2132,"(S16, S19)",(S12),0.000142,0.593595
2135,"(S16, S19, S09, S14)",(S12),0.000142,0.593595


In [107]:
#freq_itemset_fpmax = fpmax(serv_oneHot,min_support=0.0001,use_colnames=True)
association_rulesfpmax = association_rules(frequent_itemsets4, metric="support", min_threshold=0.0003, support_only = True)
fpmax_rules = association_rulesfpmax.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'lift']]
fpmax_rules

,antecedents,consequents,support,lift
0,"(S12, S18, S17, S14, S09)",(S15),0.000326,NaN
1,"(S12, S18, S17, S14, S15)",(S09),0.000326,NaN
2,"(S12, S18, S17, S09, S15)",(S14),0.000326,NaN
3,"(S12, S18, S14, S09, S15)",(S17),0.000326,NaN
4,"(S12, S17, S14, S09, S15)",(S18),0.000326,NaN
...,...,...,...,...
59,(S14),"(S12, S18, S17, S09, S15)",0.000326,NaN
60,(S09),"(S12, S18, S17, S14, S15)",0.000326,NaN
61,(S15),"(S12, S18, S17, S14, S09)",0.000326,NaN
62,(S12),(S01),0.000305,NaN


In [50]:
def predict(antecedent, rules, consequents_only = False):
    preds = rules[rules['antecedents'] == antecedent]
    if consequents_only:
        preds = preds['consequents'].apply(iter).apply(next)
    return preds

In [127]:
rules1 = association_rules(frequent_itemsets4, metric="antecedents", min_threshold=0.00001, support_only=True)
rules2 = association_rules(frequent_itemsets2, metric="antecedents", min_threshold=0.0003, support_only=True)
rules3 = association_rules(frequent_itemsets4, metric="confidence", min_threshold=0.000001, support_only=True)
rules4 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.0001, support_only=True)
#predict({"S06"}, rules1, consequents_only=False).sort_values(by='support', ascending=False)[['antecedents', 'consequents', 'support']]
#predict({"S06"}, rules2, consequents_only=False).sort_values(by='support', ascending=False)[['antecedents', 'consequents', 'support']]
#predict({"S06"}, rules3, consequents_only=False).sort_values(by='support', ascending=False)[['antecedents', 'consequents', 'support']]
predict({"S06"}, rules4, consequents_only=False).sort_values(by='support', ascending=False)[['antecedents', 'consequents', 'support']]

,antecedents,consequents,support
231,(S06),(S21),0.004582
349,(S06),(S12),0.003439
375,(S06),"(S12, S21)",0.003215
429,(S06),(S09),0.002898
525,(S06),"(S09, S21)",0.002804
...,...,...,...
14322,(S06),"(S15, S11, S14)",0.000103
14393,(S06),"(S12, S14, S09, S15, S11)",0.000101
14461,(S06),"(S12, S21, S14, S15, S11)",0.000101
14587,(S06),"(S12, S21, S14, S09, S15, S11)",0.000101


In [ ]:
After running through a few different scenarios, predict has suggested offering service S21 and S12 in additionto S6. 
The consequents that follow from these: S21, S12, and S12 + S21 are the three with highest support.